In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
import kagglehub

path = kagglehub.dataset_download("datamunge/sign-language-mnist")

Using Colab cache for faster access to the 'sign-language-mnist' dataset.


In [ ]:
import os
# Construire les chemins complets vers les fichiers
path_train = os.path.join(path, "sign_mnist_train.csv")
path_test  = os.path.join(path, "sign_mnist_test.csv")

In [ ]:
def load_sign_images(data_path):
  df = pd.read_csv(data_path)
  y = df.label.values
  # Supprimer la colonne "label" pour ne garder que les pixels
  # Convertir en tableau numpy de type float32
  # Remodeler en tableau de forme (nb_images, 28, 28, 1) → 1 canal (niveaux de gris)
  # Normaliser les pixels en divisant par 255 (valeurs entre 0 et 1)
  X = df.drop(columns=['label']).values.astype('float32').reshape((-1, 28,28, 1)) / 255.0
  return X, y

In [ ]:
Xtrain , ytrain = load_sign_images(path_train)
Xtest , ytest = load_sign_images(path_test)

In [ ]:
unique_labels = np.unique(ytrain)
unique_labels

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [ ]:
lab2idx = {
    lab: i for i , lab in enumerate(unique_labels)
}
idx2lab = {
    i: lab for lab, i in lab2idx.items()
}


In [ ]:
missing_labels_train = set(ytrain) - set(lab2idx.keys())
missing_labels_test = set(ytest) - set(lab2idx.keys())

print("Labels manquants dans y_train:", missing_labels_train)
print("Labels manquants dans y_test:", missing_labels_test)

Labels manquants dans y_train: set()
Labels manquants dans y_test: set()


In [ ]:
lab2idx

{np.int64(0): 0,
 np.int64(1): 1,
 np.int64(2): 2,
 np.int64(3): 3,
 np.int64(4): 4,
 np.int64(5): 5,
 np.int64(6): 6,
 np.int64(7): 7,
 np.int64(8): 8,
 np.int64(10): 9,
 np.int64(11): 10,
 np.int64(12): 11,
 np.int64(13): 12,
 np.int64(14): 13,
 np.int64(15): 14,
 np.int64(16): 15,
 np.int64(17): 16,
 np.int64(18): 17,
 np.int64(19): 18,
 np.int64(20): 19,
 np.int64(21): 20,
 np.int64(22): 21,
 np.int64(23): 22,
 np.int64(24): 23}

In [ ]:
ytrain = np.vectorize(lambda x: lab2idx.get(x, -1))(ytrain)
ytest = np.vectorize(lambda x: lab2idx.get(x, -1))(ytest)

In [ ]:
Xtrain, Xval , ytrain, yval = train_test_split(Xtrain, ytrain, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
data_augmentation = keras.Sequential([
    # Retourner aléatoirement horizontalement (effet miroir)
    keras.layers.RandomFlip("horizontal"),

    # Rotation aléatoire légère (±5%)
    keras.layers.RandomRotation(0.05),

    # Zoom aléatoire jusqu’à 10%
    keras.layers.RandomZoom(0.1),

    # Décalage aléatoire en x et y (±5%)
    keras.layers.RandomTranslation(0.05, 0.05)
])

In [ ]:
def build_model_cnn(input_shape=(28,28,1), num_classes=len(ytrain)):
  inputs = keras.Input(shape=(28,28,1))

  x= inputs
  x = data_augmentation(x)

  x = keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
  x = keras.layers.MaxPooling2D((2, 2))(x)

  x = keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
  x = keras.layers.MaxPooling2D((2, 2))(x)

  x = keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
  x = keras.layers.MaxPooling2D((2, 2))(x)

  x = keras.layers.Flatten()(x)

  outputs = keras.layers.Dense(num_classes, activation='softmax')(x)

  model = keras.Model(inputs=inputs, outputs=outputs)

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy']
  )

  return model


In [ ]:
model = build_model_cnn()

In [ ]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 21964)          │    25,324,492 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,417,164 (96.96 MB)

 Trainable params: 25,417,164 (96.96 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
callbacks = [
        # Réduction du learning rate quand la loss stagne
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=0.0001,
            verbose=1
        ),

        # Early stopping pour éviter l'overfitting
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),

        # Sauvegarde du meilleur modèle
        keras.callbacks.ModelCheckpoint(
            'best_model.keras',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
        ]

In [ ]:
history= model.fit(
    Xtrain, ytrain, batch_size=128,
    epochs= 50,
    validation_data= (Xval, yval),
    callbacks= callbacks

)

Epoch 1/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0726 - loss: 4.2815
Epoch 1: val_accuracy improved from -inf to 0.48789, saving model to best_model.keras
172/172 ━━━━━━━━━━━━━━━━━━━━ 16s 62ms/step - accuracy: 0.0730 - loss: 4.2750 - val_accuracy: 0.4879 - val_loss: 1.6624 - learning_rate: 0.0010
Epoch 2/50
169/172 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6513 - loss: 1.1289
Epoch 2: val_accuracy improved from 0.48789 to 0.87762, saving model to best_model.keras
172/172 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.6537 - loss: 1.1209 - val_accuracy: 0.8776 - val_loss: 0.3740 - learning_rate: 0.0010
Epoch 3/50
169/172 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9102 - loss: 0.2807
Epoch 3: val_accuracy improved from 0.87762 to 0.97232, saving model to best_model.keras
172/172 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9108 - loss: 0.2793 - val_accuracy: 0.9723 - val_loss: 0.1092 - learning_rate: 0.0010
Epoch 4/50
168/172 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/

In [ ]:
test_loss, test_acc = model.evaluate(Xtest, ytest)
print(f'loss accuracy: {test_acc}')

225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9280 - loss: 0.2802
loss accuracy: 0.9341884851455688
